<a href="https://colab.research.google.com/github/BenBreaksThings/CornettoLeague/blob/main/peer_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title **Acronyms**

"""handle identifying, sorting, and reporting acronym use"""

import re
from operator import itemgetter


class Acronyms:
	"""stores all found acronyms"""

	def __init__(self):

		self.acronym_report = []
		self.exclude_list = ['(HH:MM:SS)', '(LAG)']

	def run_line(self, line, line_num):
		"""processes each line"""
		returns = re.findall(r'\(.*?\)', str(line))

		if len(returns) > 0:
			for s in returns:
				split_str = s.split()

				good_flag = True
				for i in split_str:
					if i != '(' and i != ')' and (not i.isupper() or i in self.exclude_list):
						good_flag = False

				if good_flag:
					self.acronym_report.append([line_num, s])

	def defang_latex(self, in_string):
		"""Helper function to replace LaTeX syntax with plaintext when compiled"""
		return in_string.replace("&", "\&")

	def report_out(self):
		"""generates the report of the acronyms"""
		acronyms = len(self.acronym_report)
		if acronyms != 0:
			alrdy_report = []

			sorted(self.acronym_report, key=itemgetter(0))
			acro_set = {self.defang_latex(m[1]) for m in self.acronym_report}

			acro_list = f'{len(acro_set)} unique acronyms, used {acronyms} times:\n\n'
			acro_list += r"\noindent" + "    ".join(sorted(acro_set))
			acro_list += "\n" + r"\begin{multicols}{2}"
			acro_list += "\n" + r"\begin{itemize}[itemsep=0ex]"

			for i in self.acronym_report:
				if i[1] in alrdy_report:
					acro_list += f'\n\t\item line {i[0]:,}:  {self.defang_latex(i[1])}\t\tConsider not redefining.'
				else:
					acro_list += f'\n\t\item line {i[0]:,}:  {self.defang_latex(i[1])}'
					alrdy_report.append(i[1])
			acro_list += r"\end{itemize}\end{multicols}"

			return acro_list

		return '\nAcronym Report complete, no acronyms found.\n\n'


In [ ]:
# @title **Clarity**

"""handling the rewording and replacement of strings in a document"""

import reword_dictionary
import replace_dictionary

# TODO adapt the matching of reword and replace to regex


def find_match(string, in_list):
	"""used to find second item in list of lists"""
	for i, item in enumerate(in_list):
		try:
			j = item.index(string)
		except ValueError:
			continue
		yield i, j


class Clarity:
	"""class managing rewording of narratives"""

	def __init__(self):
		self.old_char_count = 0
		self.new_char_count = 0
		self.repl_count = 0
		self.rew_count = 0
		self.doc = []
		self.repl_summ = ''
		self.rew_summ = ''
		self.repls_found = []
		self.rewords_found = []

	def run_replacements(self):
		"""correlate to the dictionaries"""

		for index, n in enumerate(self.doc):
			for i in replace_dictionary.replace_dict:
				doc_line = self.doc[index]
				if i[0].lower() in doc_line.lower():
					x = self.doc[index].lower().count(i[0])
					self.repl_count += x
					self.old_char_count += (x * len(i[0]))
					self.new_char_count += (x * len(i[1]))
					if any(e[1] == i[0] for e in self.repls_found):
						match_tuple = list(find_match(i[0], self.repls_found))
						match_ind = match_tuple[0][0]
						self.repls_found[match_ind][0] += 1
					else:
						self.repls_found.append([x, i[0], i[1]])

		self.repl_summ = f'{self.repl_count:,} chances to streamline.\n'
		delta_chars = self.old_char_count - self.new_char_count
		all_chars_sum = sum(len(i) for i in self.doc)
		delta_pct = round((delta_chars / all_chars_sum * 100), 2)
		self.repl_summ += f'Potential character savings of {delta_chars:,}; {delta_pct}\\%\n\n'

	def run_rewords(self):
		for index, n in enumerate(self.doc):
			doc_line = self.doc[index]
			for i in reword_dictionary.reword_dict:
				if i[0].lower() in doc_line.lower():
					x = doc_line.lower().count(i[0])
					self.rew_count += x
					if any(e[1] == i[0] for e in self.rewords_found):
						match_tuple = list(find_match(i[0], self.rewords_found))
						match_ind = match_tuple[0][0]
						self.rewords_found[match_ind][0] += 1
					else:
						self.rewords_found.append([x, i[0], i[1]])

		self.rew_summ = f'{self.rew_count:,} instances that could benefit from rewording:\n\n'

	def report_out(self):
		"""generate the clarity report"""
		self.run_replacements()
		self.run_rewords()

		self.repl_summ += r"\begin{itemize}[itemsep=0ex]"
		self.repl_summ += "\n"
		for i in self.repls_found:
			self.repl_summ += f"\item {i[0]:,} instances of ``{i[1]}''; recommended change to ``{i[2]}''\n"
		self.repl_summ += r"\end{itemize}"

		self.rew_summ += r"\begin{itemize}[itemsep=0ex]"
		self.repl_summ += "\n"
		for r in self.rewords_found:
			self.rew_summ += f"\item {r[0]:,} opportunities to improve upon ``{r[1]}''; {r[2]}\n"
		self.rew_summ += r"\end{itemize}"


In [ ]:
# @title **File Handling**


"""get all files from Ingest folder for processing"""
import os
import docx


def _build_dict(my_file, file_type, my_text):
    return {
        'file': my_file,
        'type': file_type,
        'narrative': my_text
    }


def get_files(extensions):
    """callable function to get matching file types"""
    try:
        return [file for file in os.listdir("./Ingest") if file.endswith(extensions)]
    except IndexError:
        return None


class FileGather:
    """class to find and parse by file type"""

    def __init__(self):
        # TODO add a markdown file
        self.tex_files = get_files('.txt')
        self.txt_files = get_files('.tex')
        self.docx_files = get_files('.docx')
        self.tex_out = []
        self.txt_out = []
        self.docx_out = []
        self.parsing_dicts = []

        if self.tex_files:
            self.parse_tex()

        if self.txt_files:
            self.parse_txt()

        if self.docx_files:
            self.parse_doc()

    def parse_tex(self):
        """Parse .tex files"""
        for tex in self.tex_files:
            print(tex)
            full_text = []
            with open(f'./Ingest/{tex}', encoding="utf8") as tex_file:
                full_text.extend([num, line] for num, line in enumerate(tex_file))
            self.parsing_dicts.append(_build_dict(tex, '.tex', full_text))

    def parse_txt(self):
        """Parse .txt files"""
        for txt in self.txt_files:
            print(txt)
            with open(f'./Ingest/{txt}', encoding="ISO-8859-1") as txt_file:
                lines = list(txt_file.readlines())
            self.parsing_dicts.append(_build_dict(txt, '.txt', lines))

    def parse_doc(self):
        """Parse .doc and .docx files"""
        for docs in self.docx_files:
            print(docs)
            doc = docx.Document(f'./Ingest/{docs}')
            full_text = [para.text for para in doc.paragraphs]
            self.parsing_dicts.append(_build_dict(docs, '.doc', full_text))


In [ ]:
# @title **Idiom List**
idiom_list = [
  "24/7",
  "twenty-four-seven",
  "a bed of roses",
  "a bird in the hand",
  "a bird in the hand is worth two in the bush",
  "a bit much",
  "a bit too much",
  "a bite at the cherry",
  "a blessing in disguise",
  "a bolt from the blue",
  "a bone to pick",
  "a bunch of fives",
  "a burnt child dreads the fire",
  "a cat in gloves catches no mice",
  "a cat may look at a king",
  "a chain is only as strong as its weakest link",
  "a chip on one's shoulder",
  "a cinch",
  "a cut above",
  "a cut below",
  "a dime a dozen",
  "a drop in the ocean",
  "a drowning man will clutch at a straw",
  "a fish out of water",
  "a fool and his money are easily parted",
  "a friend in need is a friend indeed",
  "a golden key can open any door",
  "a good man is hard to find",
  "a hard nut to crack",
  "a house divided against itself cannot stand",
  "a house of cards",
  "a leopard can’t change his spots",
  "a little bird told me",
  "a little from column a, a little from column b",
  "a little knowledge is a dangerous thing",
  "a little of what you fancy does you good",
  "a lot on my plate",
  "a lot on one's plate",
  "a man of action",
  "a million and one",
  "a miss is as good as a mile",
  "a needle in a haystack",
  "a nod is as good as a wink",
  "a penny saved is a penny earned",
  "a picture is worth a thousand words",
  "a piece of cake",
  "a place for everything and everything in its place",
  "a quick buck",
  "a raw deal",
  "a rolling stone gathers no moss",
  "a sight for sore eyes",
  "a sitting duck",
  "a smooth sea never made a skilled sailor",
  "a snowball effect",
  "a snowball's chance in hell",
  "a stitch in time saves nine",
  "a stone's throw",
  "a storm in a teacup",
  "a tall order",
  "a trip to the sun",
  "a voice crying in the wilderness",
  "a watched pot never boils",
  "a wedge between",
  "a week is a long time in",
  "a wet blanket",
  "a whale of",
  "a whole new ball game",
  "a whole raft of",
  "abc",
  "about time",
  "about to",
  "above and beyond",
  "above board",
  "above the law",
  "above the salt",
  "above water",
  "absence makes the heart grow fonder",
  "accident of birth",
  "accident waiting to happen",
  "accidentally on purpose",
  "according to",
  "according to hoyle",
  "account for",
  "ace in the hole",
  "ace up one's sleeve",
  "ace up one’s sleeve",
  "achilles heel",
  "achilles' heel",
  "acid test",
  "acknowledge the corn",
  "acquired taste",
  "across the board",
  "across the pond",
  "act high and mighty",
  "act of congress",
  "act of god",
  "act one's age",
  "actions speak louder than words",
  "ad hoc",
  "adam’s ale",
  "add fuel to the fire",
  "add fuel to the flames",
  "add insult to injury",
  "after all",
  "after blood",
  "after one's own heart",
  "after the fact",
  "after the lord mayor's show",
  "aftermath",
  "against the clock",
  "against the grain",
  "against the run of play",
  "age before beauty",
  "ages",
  "agree to disagree",
  "agreement in principle",
  "aha moment",
  "ahead of one's time",
  "ahead of the curve",
  "ahead of the game",
  "aid and abet",
  "air rage",
  "airy fairy",
  "albatross around one's neck",
  "alive and kicking",
  "all along",
  "all and sundry",
  "all bark and no bite",
  "all bets are off",
  "all day long",
  "all dressed up and nowhere to go",
  "all ears",
  "all eyes and ears",
  "all eyes are on",
  "all fur coat and no knickers",
  "all good things must come to an end",
  "all greek to me",
  "all hands on deck",
  "all hat and no cattle",
  "all hell breaks loose",
  "all hell will let loose",
  "all in a day's work",
  "all in a day’s work",
  "all in all",
  "all in good time",
  "all in one piece",
  "all is fair in love and war",
  "all it's cracked up to be",
  "all mouth and no trousers",
  "all of a sudden",
  "all over but the shouting",
  "all over hell's half acre",
  "all over the board",
  "all over the map",
  "all over the place",
  "all rights reserved",
  "all roads lead to rome",
  "all set",
  "all singing, all dancing",
  "all sizzle and no steak",
  "all talk",
  "all that glitters is not gold",
  "all that jazz",
  "all the best",
  "all the marbles",
  "all the rage",
  "all the same",
  "all the way up",
  "all things being equal",
  "all things considered",
  "all thumbs",
  "all told",
  "all up",
  "all very well",
  "all wet",
  "alma mater",
  "along the lines of",
  "alpha and omega",
  "alter ego",
  "amateur hour",
  "amber gambler",
  "amber nectar",
  "american dream",
  "an apple a day keeps the doctor away",
  "an earful",
  "an eye for an eye",
  "an item",
  "an offer one can't refuse",
  "an ounce of prevention is worth a pound of cure",
  "ancient history",
  "and all that",
  "and change",
  "and counting",
  "and his mother",
  "and so forth",
  "and so on",
  "and the like",
  "and then some",
  "angel's advocate",
  "angle for",
  "another nail in one's coffin",
  "answer back",
  "ants in pants",
  "ants in your pants",
  "any port in a storm",
  "any tom, dick or harry",
  "anything but",
  "apple of discord",
  "apple of eye",
  "apple of one's eye",
  "apple of someone's eye",
  "apple pie order",
  "apple polisher",
  "apples and oranges",
  "apples to apples",
  "apples to oranges",
  "arm candy",
  "arm in arm",
  "armed to the teeth",
  "around the clock",
  "arrive in a body",
  "as a matter of fact",
  "as a result",
  "as american as apple pie",
  "as an aside",
  "as and when",
  "as bald as a coot",
  "as bold as brass",
  "as broad as long",
  "as busy as a bee",
  "as calm as a toad in the sun",
  "as different as chalk and cheese",
  "as easy as pie",
  "as far as",
  "as far as i can throw",
  "as fit as a fiddle",
  "as for",
  "as long as",
  "as mad as a hatter",
  "as nice as ninepence",
  "as pale as a ghost",
  "as pale as death",
  "as poor as a church mouse",
  "as red as a cherry",
  "as right as rain",
  "as simple as that",
  "as soon as",
  "as the crow flies",
  "as thick as thieves",
  "as to",
  "as well",
  "as well as",
  "ask out",
  "asleep at the wheel",
  "at a discount",
  "at bay",
  "at bottom",
  "at cross purposes",
  "at daggers drawn",
  "at death's door",
  "at death’s door",
  "at each other's throats",
  "at loggerheads",
  "at odds",
  "at sea",
  "at sixes and sevens",
  "at stake",
  "at straws",
  "at the bit",
  "at the drop of a dime",
  "at the drop of a hat",
  "at the eleventh hour",
  "at the end of one's rope",
  "at the end of the day",
  "at the helm",
  "at the same time",
  "at wit's end",
  "avant-garde",
  "axe to grind",
  "babe in arms",
  "babe in the woods",
  "babe magnet",
  "baby blues",
  "baby boomer",
  "babysitter test",
  "back against the wall",
  "back and forth",
  "back at you",
  "back burner",
  "back door",
  "back forty",
  "back in the day",
  "back of beyond",
  "back office",
  "back the wrong horse",
  "back to basics",
  "back to square one",
  "back to the drawing board",
  "back to the wall",
  "backhanded compliment",
  "backing and filling",
  "backroom boy(s)",
  "backseat driver",
  "bad apple",
  "bad blood",
  "bad egg",
  "bad taste in one's mouth",
  "bad-mouth",
  "bag and baggage",
  "bag of tricks",
  "bail out",
  "baker’s dozen",
  "balancing act",
  "ball and chain",
  "ballpark figure",
  "banana republic",
  "bang for the buck",
  "bang for your buck",
  "bang head against wall",
  "bang on",
  "bang one's head",
  "banker’s hours",
  "banner year",
  "baptism by fire",
  "bar fly",
  "bare bones",
  "bare one's heart",
  "bare one's soul",
  "barfly",
  "bark up the wrong tree",
  "barking dogs seldom bite",
  "barking up the wrong tree",
  "barrel of laughs",
  "barrel of monkeys",
  "basket case",
  "bat an eye",
  "bated breath",
  "batten down the hatches",
  "bay for blood",
  "be in the same boat",
  "be up on",
  "bean counters",
  "bear fruit",
  "bear the brunt of",
  "beat a dead horse",
  "beat a hasty retreat",
  "beat around the bush",
  "beat someone to the draw",
  "beat someone to the punch",
  "beat the air",
  "beat the drum",
  "beat the heat",
  "beau monde",
  "beauty is in the eye of the beholder",
  "beauty is only skin deep",
  "beck and call",
  "bed of roses",
  "bedroom eyes",
  "bee in bonnet",
  "bee in one's bonnet",
  "beef up",
  "been around",
  "before one can say jack robinson",
  "beg to differ",
  "beggar thy neighbor",
  "beggars can’t be choosers",
  "behind bars",
  "behind closed doors",
  "behind the scenes",
  "behind the times",
  "being in love",
  "bell the cat",
  "belle of the ball",
  "bells and whistles",
  "belly laugh",
  "belly up",
  "below the belt",
  "below the radar",
  "bend an elbow",
  "bend over backwards",
  "bent out of shape",
  "best of both worlds",
  "best thing since sliced bread",
  "bet the farm",
  "better half",
  "better late than never",
  "better off",
  "between a rock and a hard place",
  "between the devil and the deep blue sea",
  "between you and me",
  "betwixt and between",
  "beyond a shadow of doubt",
  "beyond the pale",
  "beyond the shadow of a doubt",
  "beyond the veil",
  "bide time",
  "big apple",
  "big brother",
  "big cheese",
  "big deal",
  "big fish",
  "big gun",
  "big mouth",
  "big picture",
  "big time",
  "bigger fish to fry",
  "bill of goods",
  "bird of passage",
  "bird's-eye view",
  "birds of a feather",
  "birds of a feather flock together",
  "bird’s eye view",
  "bite head off",
  "bite off more than can chew",
  "bite off more than you can chew",
  "bite the bullet",
  "bite the dust",
  "bite the hand that feed",
  "bite the hand that feeds you",
  "bite tongue",
  "bits and pieces",
  "bitter medicine",
  "bitter pill",
  "bitter pill to swallow",
  "black and blue",
  "black and white",
  "black book",
  "black day",
  "black eye",
  "black friday",
  "black hole",
  "black mood",
  "black out",
  "black sheep",
  "black-and-blue",
  "blank check",
  "blast from the past",
  "blaze a trail",
  "blazing row",
  "blessing in disguise",
  "blind alley",
  "blind as a bat",
  "blind date",
  "blind impulse",
  "blind spot",
  "blinded by love",
  "blood and thunder",
  "blood is thicker than water",
  "blood on hands",
  "blood, sweat, and tears",
  "blow away",
  "blow away the cobwebs",
  "blow cover",
  "blow hot and cold",
  "blow mind",
  "blow off steam",
  "blow one's stack",
  "blow one's top",
  "blow the whistle",
  "blow up",
  "blow your own trumpet",
  "blow-by-blow account",
  "blowing smoke",
  "blue blood",
  "blue eyed boy",
  "blue light special",
  "blue-collar",
  "bob's your uncle",
  "bob’s your uncle",
  "body and soul",
  "boil the ocean",
  "bolt from the blue",
  "bon appetit",
  "bona fide",
  "bone dry",
  "bone of contention",
  "bone to pick",
  "born on the wrong side of the blanket",
  "born with a silver spoon in mouth",
  "born yesterday",
  "borrow trouble",
  "bosom friend",
  "bottom dollar",
  "bottom line",
  "bottom of the barrel",
  "boy toy",
  "boys will be boys",
  "brain drain",
  "brainstorm",
  "bread and butter",
  "break a leg",
  "break down",
  "break fresh ground",
  "break new ground",
  "break one’s back",
  "break open",
  "break out in a cold sweat",
  "break the bank",
  "break the ice",
  "break the mould",
  "break the news",
  "break up",
  "breakout",
  "bright spark",
  "bring home the bacon",
  "bring owls to athens",
  "bring someone to book",
  "bring to the table",
  "bring up",
  "bringing a knife to a gunfight",
  "broken heart",
  "brought forward",
  "brown study",
  "brownie points",
  "brush under the carpet",
  "buck up",
  "bucket list",
  "buckle down",
  "buffer zone",
  "bug someone",
  "bug-eyed",
  "build bridges",
  "building blocks",
  "bull by the horns",
  "bull in a china shop",
  "bull’s eye",
  "bump in the road",
  "bump into",
  "bun in the oven",
  "bundle of nerves",
  "bundle up",
  "burn a bridge",
  "burn bridges",
  "burn fingers",
  "burn one's bridges",
  "burn the candle at both ends",
  "burn the midnight oil",
  "burn your bridges",
  "burst at the seams",
  "burst the bubble",
  "bury head in the sand",
  "bury the hatchet",
  "business as usual",
  "busman's holiday",
  "busted flush",
  "busy as a beaver",
  "busy bee",
  "butter up",
  "butter wouldn't melt",
  "butterflies in stomach",
  "button your lip",
  "buy a pig in a poke",
  "buy the farm",
  "buy time",
  "buzz word",
  "by a whisker",
  "by all means",
  "by and by",
  "by chance",
  "by dint of",
  "by far",
  "by hook or by crook",
  "by the skin of one's teeth",
  "by the skin of your teeth",
  "by the way",
  "by word of mouth",
  "cake and eat it too",
  "cakewalk",
  "call a spade a spade",
  "call bluff",
  "call it a day",
  "call it a night",
  "call names",
  "call of duty",
  "call of the wild",
  "call on",
  "call the shots",
  "call the tune",
  "calm before the storm",
  "camp follower",
  "can of worms",
  "can say that again",
  "can't make an omelet",
  "can't make an omelette",
  "can't see the forest for the trees",
  "can’t judge a book by its cover",
  "can’t stand the sight of",
  "card up sleeve",
  "carrot and stick",
  "carrot-and-stick",
  "carry a torch",
  "carry coals to newcastle",
  "carry on",
  "carry out",
  "carry the can",
  "carte blanche",
  "cash in",
  "cash in one's chips",
  "cash-strapped",
  "cast aspersions",
  "cast the first stone",
  "castle in the air",
  "castles in the air",
  "cat among the pigeons",
  "cat fight",
  "cat got your tongue",
  "cat nap",
  "cat's paw",
  "cat-and-mouse",
  "catch a cold",
  "catch eye",
  "catch more flies with honey",
  "catch napping",
  "catch one's death of cold",
  "catch some rays",
  "catch someone's eye",
  "catch up",
  "catch-22",
  "catnap",
  "caught between two stools",
  "caught in the crossfire",
  "caught red-handed",
  "cave in",
  "champagne taste on a beer budget",
  "change horses in midstream",
  "change of heart",
  "change one's tune",
  "changing of the guard",
  "chapter and verse",
  "charity begins at home",
  "charley horse",
  "chase rainbows",
  "chattering classes",
  "cheap shot",
  "cheat sheet",
  "cherry-pick",
  "chew the fat",
  "chicken hearted",
  "chickens come home to roost",
  "child's play",
  "child’s play",
  "chill out",
  "chill pill",
  "chill to the bone",
  "chill to the marrow",
  "chime in",
  "chin music",
  "chin up",
  "chink in the armour",
  "chip off the old block",
  "chip on shoulder",
  "choke up",
  "chomp at the bit",
  "chop and change",
  "chop chop",
  "chop shop",
  "chow down",
  "chuck a wobbly",
  "circle the wagons",
  "claim to fame",
  "clam up",
  "clarion call",
  "class act",
  "class clown",
  "clean slate",
  "clean sweep",
  "clean up nicely",
  "clear the air",
  "clear the decks",
  "clip someone's wings",
  "cloak and dagger",
  "close call",
  "close shave",
  "close the books",
  "close, but no cigar",
  "closed book",
  "cloud nine",
  "coast is clear",
  "cock and bull story",
  "cock-a-hoop",
  "code of conduct",
  "coin a phrase",
  "coin money",
  "cold comfort",
  "cold day in hell",
  "cold day in july",
  "cold feet",
  "cold fish",
  "cold hearted",
  "cold shoulder",
  "cold sweat",
  "cold turkey",
  "collision course",
  "comb through",
  "come a cropper",
  "come across",
  "come again",
  "come by something honestly",
  "come clean",
  "come down with",
  "come full circle",
  "come hell or high water",
  "come of age",
  "come on",
  "come out",
  "come out in the wash",
  "come out of the closet",
  "come out swinging",
  "come over",
  "come rain and shine",
  "come rain or shine",
  "come to a head",
  "come to a pretty pass",
  "come to blows",
  "come to grief",
  "come to grips with",
  "come to pass",
  "come to think of/about it",
  "come up",
  "come up with",
  "come what may",
  "comes to the crunch",
  "coming down the pike",
  "comparing apples to oranges",
  "concern with",
  "conspiracy of silence",
  "cook someone's goose",
  "cook up a storm",
  "cooking up a storm",
  "cool as a cucumber",
  "cool as cucumber",
  "cool cat",
  "cool heels",
  "cool your heels",
  "cool your jets",
  "copycat",
  "cost an arm and a leg",
  "costs an arm and a leg",
  "couch potato",
  "couldn’t care less",
  "count chicken",
  "count on",
  "cover one’s tracks",
  "crack of dawn",
  "crack the whip",
  "crack up",
  "crash a party",
  "crash course",
  "creme de la creme",
  "crickets",
  "crocodile tears",
  "cross paths",
  "cross swords",
  "cross that bridge",
  "cross the bridge",
  "cross the line",
  "cross to bear",
  "crowning glory",
  "crunch the numbers",
  "crunch time",
  "crush",
  "cry me a river",
  "cry over spilt",
  "cry over spilt milk",
  "cry wolf",
  "cry your eyes out",
  "crystal clear",
  "cup of joe",
  "cup of tea",
  "cupboard love",
  "curiosity killed the cat",
  "curry favour",
  "cut and dried",
  "cut and thrust",
  "cut both ways",
  "cut corners",
  "cut down",
  "cut from the same cloth",
  "cut it fine",
  "cut off your nose to spite your face",
  "cut somebody some slack",
  "cut someone some slack",
  "cut the cord",
  "cut the crap",
  "cut the gordian knot",
  "cut the mustard",
  "cut to the chase",
  "cut your coat according to your cloth",
  "cut your teeth on something",
  "cutting corners",
  "cutting edge",
  "cutting-edge",
  "dab hand",
  "damp squib",
  "dance to someone's tune",
  "dance to tune",
  "dance with the devil",
  "dark horse",
  "darken someone's door",
  "day in, day out",
  "day late and a dollar short",
  "day of reckoning",
  "de facto",
  "de jure",
  "dead ahead",
  "dead and buried",
  "dead as a dodo",
  "dead as a doornail",
  "dead as the dodo",
  "dead end",
  "dead eye",
  "dead heat",
  "dead in the water",
  "dead of winter",
  "dead ringer",
  "dead run",
  "dead shot",
  "deals with",
  "death knell",
  "deep dive",
  "deep pockets",
  "deja vu",
  "deliver the goods",
  "devil is in the details",
  "devil may care",
  "devil's advocate",
  "devil’s advocate",
  "diamond in the rough",
  "diamond is forever",
  "die hard",
  "die in harness",
  "dig heels in",
  "dig one’s own grave",
  "dig up dirt",
  "dime a dozen",
  "dirt cheap",
  "dirty look",
  "discretion is the greater part of valour",
  "dish fit for the gods",
  "divide and conquer (or rule)",
  "do 12-ounce curls",
  "do justice to",
  "do one’s bit",
  "do or die",
  "do something at the drop of a hat",
  "do the trick",
  "do unto others as you would have them do unto you",
  "dodge a bullet",
  "does exactly what it says on the tin",
  "doesn't amount to a hill of beans",
  "dog and pony show",
  "dog days",
  "dog days of summer",
  "dog days of the summer",
  "dog eat dog",
  "dog in the",
  "dog in the manger",
  "dog tired",
  "dog-and-pony show",
  "dog-eat-dog",
  "dog’s life",
  "doldrums",
  "dollars to donuts",
  "don't beat a dead horse",
  "don't count your chickens before they hatch",
  "don't cry over spilled milk",
  "don't cry over spilt milk",
  "don't give up your day job",
  "don't judge a book by its cover",
  "don't look a gift horse in the mouth",
  "don't put all your eggs in one basket",
  "done for",
  "done in",
  "don’t count your chickens before they hatch",
  "don’t give up the day job",
  "don’t look a gift horse in the mouth",
  "don’t put all your eggs in one basket",
  "doozy",
  "dos and don’ts",
  "dot the i’s and cross the t’s",
  "double date",
  "double take",
  "double whammy",
  "double-dip",
  "double-edged sword",
  "doubting thomas",
  "down and out",
  "down for the count",
  "down in the dumps",
  "down the drain",
  "down the road",
  "down to earth",
  "down to the wire",
  "drag feet (or heels)",
  "drag one's feet",
  "drag one's heels",
  "drain the lizard",
  "drain the swamp",
  "drastic times call for drastic measures",
  "draw a blank",
  "draw a line under",
  "draw a long bow",
  "draw the line",
  "dream come true",
  "dressed to kill",
  "dribs and drabs",
  "drink like a fish",
  "drive crazy",
  "drive home",
  "drive up the wall",
  "driving me nuts",
  "drop a bombshell",
  "drop a line",
  "drop by",
  "drop dead",
  "drop like flies",
  "drop off",
  "drop the ball",
  "dropped a line",
  "dropped the ball",
  "dry run",
  "ducks and drakes",
  "dutch courage",
  "dutch uncle",
  "dyed-in-the-wool",
  "eager beaver",
  "eagle-eyed",
  "early bird",
  "early bird gets the worm",
  "early days",
  "earworm",
  "easier said than done",
  "easter egg",
  "easy come, easy go",
  "easy does it",
  "easy money",
  "easy peasy",
  "eat crow",
  "eat humble pie",
  "eat like a horse",
  "eat someone's lunch",
  "eat words",
  "eat your heart out",
  "egg on your face",
  "eighty-six",
  "elephant in the room",
  "elevator music",
  "elevator pitch",
  "eleventh hour",
  "elvis has left the building",
  "end of story",
  "end of the day",
  "end up",
  "even steven",
  "every cloud has a silver lining",
  "every dog has",
  "every man for himself",
  "every man jack",
  "every nook and cranny",
  "every picture tells a story",
  "everything but the kitchen sink",
  "excused boots",
  "eye candy",
  "eye to eye",
  "face the music",
  "face to face",
  "faint of heart",
  "fair and square",
  "fair-weather friend",
  "fake it till you make it",
  "fall between the cracks",
  "fall for",
  "fall for something",
  "fall from grace",
  "fall in love",
  "fall in love with somebody",
  "fall on deaf ears",
  "fall on one's sword",
  "fall out",
  "fall prey to",
  "familiarity breeds contempt",
  "famous last words",
  "fan the flames",
  "fancy free",
  "fancy someone",
  "far cry from",
  "far-fetched",
  "fashion-forward",
  "fast food",
  "fat cat",
  "fat chance",
  "fate worse than death",
  "father figure",
  "faux pas",
  "feast your eyes on",
  "feather in one's cap",
  "feather in your cap",
  "feather one's nest",
  "feather one's own nest",
  "feather one’s (own) nest",
  "fed up with",
  "feel blue",
  "feel like a million dollars",
  "feel on top of the world",
  "feel the pinch",
  "feet of clay",
  "fell off a truck",
  "fell off the back of a lorry",
  "few sandwiches short of a picnic",
  "fiddle while rome burns",
  "field day",
  "fifteen minutes of fame",
  "fifth wheel",
  "fight fire with fire",
  "fight like cat and dog",
  "fight tooth and nail",
  "figure out",
  "fill the bill",
  "filthy rich",
  "find feet",
  "find one's voice",
  "find your feet",
  "finger-pointing",
  "fingerprints are all over",
  "fingers crossed",
  "fingers to the bone",
  "finishing touch",
  "fire in the belly",
  "first in, best dressed",
  "first light",
  "first things first",
  "fish for a compliment",
  "fish for compliments",
  "fish or cut bait",
  "fish out of water",
  "fish rots from the head down",
  "fit as a fiddle",
  "fit of anger",
  "five-finger discount",
  "flash in the pan",
  "flat broke",
  "flat out like a lizard",
  "flesh and blood",
  "flew the coop",
  "flip flop",
  "flip the bird",
  "flip-flop",
  "flog a dead horse",
  "fly by the seat of one's pants",
  "fly high",
  "fly in the ointment",
  "fly off the handle",
  "fly on the wall",
  "foam at the mouth",
  "follow by",
  "follow in someone's footsteps",
  "follow your heart",
  "food for thought",
  "fool’s paradise",
  "foot in the door",
  "foot the bill",
  "footloose and fancy-free",
  "for a song",
  "for a while",
  "for crying out loud",
  "for xyz reasons",
  "forbidden fruit is always the sweetest",
  "fortune favours the bold",
  "forty winks",
  "foul play",
  "fourth estate",
  "fox in the chickenhouse",
  "fox in the henhouse",
  "freak out",
  "free spirit",
  "french leave",
  "freudian slip",
  "from a to z",
  "from cradle to grave",
  "from now on",
  "from pillar to post",
  "from scratch",
  "from soup to nuts",
  "from the bottom of my heart",
  "from the bottom of one's heart",
  "from the horse's mouth",
  "from time to time",
  "fubar",
  "full cry",
  "full fathom five",
  "full of beans",
  "full of the joys of spring",
  "game of chicken",
  "get a charley horse",
  "get a life",
  "get a second wind",
  "get a taste of your own medicine",
  "get a word in edgewise",
  "get act together",
  "get along with",
  "get bent out of shape",
  "get carried away",
  "get cold feet",
  "get cracking",
  "get down to brass tacks",
  "get hitched",
  "get in on the ground floor",
  "get in shape",
  "get off",
  "get off scot free",
  "get on nerves",
  "get one's ducks in a row",
  "get one's hands dirty",
  "get one’s feet wet",
  "get out of bed on the wrong side",
  "get out of dodge",
  "get out of hand",
  "get over it",
  "get rid of",
  "get someone's goat",
  "get something out of your system",
  "get the ball rolling",
  "get the picture",
  "get the runaround",
  "get the sack (boot, axe)",
  "get the sack, be sacked",
  "get the third degree",
  "get the wrong end of the stick",
  "get through",
  "get to grips with",
  "get to know",
  "get to the bottom",
  "get under skin",
  "get wind of",
  "get wind of something",
  "get with the program",
  "get your act together",
  "get your goat",
  "ginger up",
  "give 'em hell",
  "give a shot",
  "give airs",
  "give and take",
  "give hell",
  "give it a whirl",
  "give someone the benefit of the doubt",
  "give someone the cold shoulder",
  "give someone the old heave-ho",
  "give something a whirl",
  "give the devil his due",
  "give the green light",
  "give the slip",
  "give up",
  "glass ceiling",
  "go after",
  "go along with",
  "go ape",
  "go back to the drawing board",
  "go ballistic",
  "go bananas",
  "go belly up",
  "go berserk",
  "go bonkers",
  "go by the book",
  "go cold turkey",
  "go down in flames",
  "go down like a lead balloon",
  "go for broke",
  "go mental",
  "go nuclear",
  "go nuts",
  "go off half-cocked",
  "go off the deep end",
  "go off the rails",
  "go on a wild goose chase",
  "go out on a limb",
  "go out with",
  "go pear-shaped",
  "go round in circles",
  "go see a man about a dog",
  "go steady",
  "go the extra mile",
  "go through a rough patch",
  "go through channels",
  "go to the dogs",
  "go to the mattresses",
  "go under the knife",
  "go viral",
  "go with the flow",
  "god helps those who help themselves",
  "going south",
  "going to hell in a handbasket",
  "good account of",
  "good deal",
  "good egg",
  "good faith",
  "good for the goose is good for the gander",
  "good grief",
  "good samaritan",
  "good things come to those who wait",
  "goody two shoes",
  "goose egg",
  "grass is always greener on the other side",
  "grease monkey",
  "grease the wheels",
  "greasy spoon",
  "great deal",
  "great minds think alike",
  "great scott",
  "green around the gills",
  "green as grass",
  "green thumb",
  "green with envy",
  "greenhorn",
  "grind one's teeth",
  "grist to the mill",
  "grit one’s teeth",
  "guess is as good as mine",
  "guilty pleasure",
  "guinea pig",
  "gut feeling",
  "had it up to here",
  "hail mary",
  "hair of the dog",
  "hair's breadth",
  "hakuna matata",
  "half baked",
  "halfway house",
  "hand in hand",
  "hands are tied",
  "hands down",
  "hang in there",
  "hang it up",
  "hang on",
  "hang tough",
  "hang up",
  "hanging by a thread",
  "happy as a clam",
  "happy go lucky",
  "happy wife happy life",
  "happy-go-lucky",
  "hard and fast",
  "hard bargain",
  "hard cheese",
  "hard nut to crack",
  "harp on",
  "has the cat got your tongue?",
  "has-been",
  "haste makes waste",
  "hat trick",
  "hatchet job",
  "haul over the coals",
  "have a ball",
  "have a cow",
  "have a lead foot",
  "have a lot on one's plate",
  "have a lot riding on",
  "have a nose for",
  "have designs on",
  "have eyes only for",
  "have hand of aces",
  "have it out with",
  "have never had it so good",
  "have no business",
  "have no truck with",
  "have the hots for",
  "have your say",
  "head above water",
  "head and shoulders",
  "head and shoulders above",
  "head in the clouds",
  "head in the sand",
  "head over heels",
  "head south",
  "head start",
  "head-on",
  "headed south",
  "heading south",
  "heads are going to roll",
  "heads up",
  "heads will roll",
  "hear on the grapevine",
  "heart and soul",
  "heart goes out to",
  "heart touching",
  "heavens open",
  "heavy heart",
  "heavy hitter",
  "helicopter parenting",
  "hell for leather",
  "henpecked",
  "herculean task",
  "hide light under a bushel",
  "high as a kite",
  "high five",
  "high on the hog",
  "high roller",
  "high time",
  "high up",
  "hightail it",
  "highways and byways",
  "hissy fit",
  "hit a wall",
  "hit it out of the park",
  "hit or miss",
  "hit the books",
  "hit the ground running",
  "hit the hay",
  "hit the jackpot",
  "hit the nail on the head",
  "hit the panic button",
  "hit the road",
  "hit the roof",
  "hit the sack",
  "hit the spot",
  "hive mind",
  "hive of activity",
  "hobson's choice",
  "hobson’s choice",
  "hold all the aces",
  "hold one's liquor",
  "hold one's peace",
  "hold the phone",
  "hold tongue",
  "hold your feet to the fire",
  "hold your horses",
  "home away from home",
  "home bird",
  "home is where the heart is",
  "home truths",
  "honesty is the best policy",
  "honor system",
  "hook, line, and sinker",
  "hope for the best, but prepare for the worst",
  "hot air",
  "hot mess",
  "hot on the heels",
  "hot potato",
  "household name",
  "how come",
  "hundred and ten percent",
  "i could eat a horse",
  "icing on the cake",
  "idle hands are the devil’s tools",
  "if it ain’t broke, don’t fix it",
  "if it had been a snake, it would have bitten",
  "if it’s not one thing, it’s the other",
  "if it’s the last thing i do",
  "if the shoe fits, wear it",
  "if wishes were horses, beggars would ride",
  "if you scratch my back, i’ll scratch yours",
  "ignorance is bliss",
  "in a bad way",
  "in a clover",
  "in a fog",
  "in a heartbeat",
  "in a huff",
  "in a jam",
  "in a new york minute",
  "in a nutshell",
  "in a pickle",
  "in a pinch",
  "in a row",
  "in a rut",
  "in a trice",
  "in at the deep end",
  "in bed with",
  "in black and white",
  "in broad daylight",
  "in clover",
  "in cold blood",
  "in deep water",
  "in dire straits or need",
  "in due course",
  "in for a penny, in for a pound",
  "in full swing",
  "in hand",
  "in high dudgeon",
  "in his cross hairs",
  "in his crosshairs",
  "in his cups",
  "in hot water",
  "in lieu of",
  "in light of",
  "in limbo",
  "in mint condition",
  "in no time",
  "in one fell swoop",
  "in one's element",
  "in one’s blood",
  "in short order",
  "in someone's wheelhouse",
  "in stitches",
  "in tandem",
  "in the air",
  "in the ascendant",
  "in the bad books",
  "in the bag",
  "in the black",
  "in the blink of an eye",
  "in the bud",
  "in the buff",
  "in the cahoots",
  "in the can",
  "in the cards",
  "in the cross hairs",
  "in the crosshairs",
  "in the dark",
  "in the dock",
  "in the driver's seat",
  "in the driver’s seat",
  "in the good books",
  "in the heat of the moment",
  "in the hot seat",
  "in the interim",
  "in the limelight",
  "in the limelight, in the spotlight",
  "in the long run",
  "in the loop",
  "in the meantime",
  "in the nick of time",
  "in the pink",
  "in the pipeline",
  "in the red",
  "in the same boat",
  "in the toilet",
  "in the works",
  "in touch",
  "in two minds",
  "in vain",
  "in your face",
  "iron out",
  "it in the bag",
  "it is a poor workman who blames his tools",
  "it is always darkest before the dawn",
  "it never rains but it pours",
  "it takes one to know one",
  "it takes two to tango",
  "it won't fly",
  "it's a piece of cake",
  "it's a wash",
  "it's all greek to me",
  "it's no skin off my back",
  "it's no skin off my nose",
  "it's not over till the fat lady sings",
  "it's not rocket science",
  "it's raining cats and dogs",
  "itchy feet",
  "it’s a small world",
  "it’s anyone’s call",
  "it’s better to be safe than sorry",
  "it’s not brain surgery",
  "i’ll eat my hat",
  "jack of all trades",
  "jam session",
  "jig is up",
  "jim crow",
  "john doe",
  "join the club",
  "joined at the hip",
  "jot or tittle",
  "judge a book by its cover",
  "jump in with both feet",
  "jump on the bandwagon",
  "jump the gun",
  "jump the shark",
  "jump the track",
  "jump through hoops",
  "jump to conclusions",
  "just around the corner",
  "just deserts",
  "just for the record",
  "just in case",
  "just what the doctor ordered",
  "kangaroo court",
  "keen as mustard",
  "keep a stiff upper lip",
  "keep an ear to the ground",
  "keep an eye on",
  "keep an eye peeled",
  "keep at arm’s length",
  "keep body and soul together",
  "keep card close to chest",
  "keep it under your hat",
  "keep on top",
  "keep one’s own counsel",
  "keep promise",
  "keep schtum",
  "keep someone at arm's length",
  "keep something at bay",
  "keep the wolf from the door",
  "keep your chin up",
  "keep your eyes peeled",
  "keep your friends close and your enemies closer",
  "keep your hair on",
  "keep your nose clean",
  "keep your pants on",
  "keep your powder dry",
  "keeping one's nose to the grindstone",
  "kick ass, kick butt",
  "kick the bucket",
  "kick the can down the road",
  "kick the habit",
  "kill a fly with an elephant gun",
  "kill the goose that laid the golden egg",
  "kill the goose that lays the golden egg",
  "kill time",
  "kill two birds with one stone",
  "king of the hill",
  "kink in one's neck",
  "kiss and make up",
  "kith and kin",
  "knee jerk reaction",
  "knight in shining armour",
  "knock on wood",
  "knock on wood (touch wood)",
  "knock out",
  "knock socks off",
  "knock some sense into",
  "knock someone's socks off",
  "knock up",
  "knockout",
  "knotty problem",
  "know on which side bread is buttered",
  "know the drill",
  "know the ropes",
  "know which side one’s bread is buttered",
  "know which way the wind is blowing",
  "labour of love",
  "lame duck",
  "larger than life",
  "last but not least",
  "last laugh",
  "last long",
  "late unpleasantness",
  "laughing stock",
  "laughs last laughs loudest",
  "laughs last laughs the loudest",
  "laughs last, laughs loudest",
  "laughs last, laughs the loudest",
  "laughter is the best medicine",
  "lead a horse to water",
  "lead astray",
  "lean towards",
  "leaps and bounds",
  "learn the ropes",
  "learn to walk before you run",
  "learning curve",
  "learning is a dangerous thing",
  "leave at the altar",
  "leave no stone unturned",
  "leave someone in the lurch",
  "left out in the cold",
  "leg it",
  "leg up",
  "lemon",
  "lend a hand",
  "lend an ear",
  "lend me your ears",
  "let bygones be bygones",
  "let off steam",
  "let one's hair down",
  "let sleeping dogs lie",
  "let someone off the hook",
  "let the cat out of the bag",
  "let the chips fall where they may",
  "let the genie out of the bottle",
  "let your heart rule your head",
  "letter of the law",
  "level playing field",
  "lick one's wounds",
  "lie low",
  "life is a bowl of cherries",
  "life of it's own",
  "life of its own",
  "light a fire under someone",
  "light at the end of the tunnel",
  "light of one’s life",
  "light years away",
  "lighten up",
  "lightning fast",
  "lights are on but nobody is home",
  "like a bat out of hell",
  "like a cat on a hot tin roof",
  "like a chicken with its head cut off",
  "like a duck to water",
  "like a hole in the head",
  "like a kid in a candy store",
  "like a moth to a flame",
  "like chalk and cheese",
  "like father like son",
  "like father, like son",
  "like riding a bicycle",
  "like shooting",
  "like taking candy from a baby",
  "like the back of one's hand",
  "like the cat that got the cream",
  "like the wind",
  "like two peas in a pod",
  "lily-livered",
  "line in the sand",
  "lion's den",
  "lion's share",
  "lion’s share",
  "lip service",
  "liquor up",
  "live and breathe",
  "live and learn",
  "live at rack and manger",
  "live large",
  "live on the edge",
  "live wire",
  "living in cloud cuckooland",
  "living on borrowed time",
  "living under a rock",
  "loaded for bear",
  "loan shark",
  "lock horns",
  "lone it",
  "lone wolf",
  "long arm of the law",
  "long in the tooth",
  "long run",
  "long shot",
  "long time no see",
  "look before you leap",
  "look forward",
  "look on the bright side",
  "look the other way",
  "look what the cat dragged in",
  "loose cannon",
  "lose head",
  "lose it",
  "lose one's touch",
  "lose the thread",
  "lose touch",
  "lose your marbles",
  "lost in thought",
  "lost on",
  "lost to",
  "love at first sight",
  "love child",
  "love is blind",
  "love me, love my dog",
  "love nest",
  "love of life",
  "love rat",
  "love someone with all of one's heart and soul",
  "love to death",
  "lovey-dovey",
  "lovey–dovey",
  "low hanging fruit",
  "low-hanging fruit",
  "lower the bar",
  "lower the boom",
  "mad as a box of",
  "mad as a hatter",
  "mad at",
  "made it",
  "main squeeze",
  "make a beeline for",
  "make a break for it",
  "make a clean breast",
  "make a long story short",
  "make a mountain out of a molehill",
  "make a silk purse out of a sow's ear",
  "make a virtue of necessity",
  "make all the difference",
  "make an ass",
  "make an omelet without breaking some eggs",
  "make common cause",
  "make do",
  "make ends meet",
  "make fish of one and fowl of the other",
  "make hay",
  "make hay while the sun shines",
  "make it big",
  "make light of",
  "make like a tree and leave",
  "make love",
  "make nice",
  "make no bones about",
  "make one's mark",
  "make sense",
  "make someone's day",
  "make the best of",
  "make waves",
  "man cave",
  "man of straw",
  "many hands make light work",
  "man’s best friend",
  "march to the beat of your own drum",
  "mark my words",
  "marry in haste, repent at leisure",
  "match made in heaven",
  "may-december",
  "may-december marriage",
  "me time",
  "mean business",
  "means to an end",
  "medusa tattoo",
  "meet halfway",
  "meeting of the minds",
  "melting pot",
  "memorial day",
  "memory like a sieve",
  "mend fences",
  "method to madness",
  "mileage may vary",
  "miles away",
  "miles to go before i sleep",
  "mind one's p's and q's",
  "mind your own beeswax",
  "mind your own business",
  "miss the boat",
  "monday morning quarterback",
  "money doesn’t grow on trees",
  "monkey business",
  "month of sundays",
  "more fun than a barrel of monkeys",
  "more than meets the eye",
  "more things change, the more they stay the same",
  "mother nature",
  "move heaven and earth",
  "move the needle",
  "move up in the world",
  "movers and shakers",
  "mr right, ms right",
  "much of a muchness",
  "mull over",
  "mum's the word",
  "mumbo jumbo",
  "mum’s the word",
  "music to ears",
  "music to my ears",
  "must have",
  "mutton dressed up as lamb",
  "my dogs are barking",
  "my old man, my old lady",
  "my way or the highway",
  "nail biter",
  "nail in the coffin",
  "nail-biter",
  "name is mud",
  "necessary evil",
  "neck and neck",
  "neck of the woods",
  "nepo baby",
  "nest egg",
  "never in a million years",
  "never in wildest dreams",
  "never look a gift horse in the mouth",
  "never mind",
  "never say die",
  "new blood",
  "new kid on the block",
  "new wrinkle",
  "new york minute",
  "next door",
  "next of kin",
  "nice chunk of change",
  "nickel and dime",
  "night owl",
  "nightmare dressed like a daydream",
  "nine lives",
  "nine times out of ten",
  "nine-to-five job",
  "nip in the bud",
  "no dice",
  "no feeling is final",
  "no holds barred",
  "no love lost",
  "no love lost between",
  "no man is an island",
  "no names, no pack drill",
  "no news is good news",
  "no pain, no gain",
  "no picnic",
  "no rhyme or reason",
  "no room to swing a cat",
  "no shit, sherlock",
  "no spring chicken",
  "no tree grows to the sky",
  "none of your business",
  "none of your lip",
  "nose in the air",
  "not all it’s cracked up to be",
  "not at all",
  "not believe eyes",
  "not cut out for",
  "not enough room to swing a cat",
  "not give a damn",
  "not give a fig",
  "not have a cat in hell's chance",
  "not have a clue",
  "not have a prayer",
  "not hurt a fly or flea",
  "not know jack",
  "not lift a finger",
  "not mince words",
  "not one's cup of tea",
  "not playing with a full deck",
  "not ready for prime time",
  "not sit well with",
  "not the sharpest tool in the shed",
  "notch above",
  "nothing to write home about",
  "nowhere near",
  "number is up",
  "nuts and bolts",
  "nutty as a fruitcake",
  "of course",
  "off grid",
  "off his rocker",
  "off on the wrong foot",
  "off the chain",
  "off the cuff",
  "off the hook",
  "off the record",
  "off the wagon",
  "old habits die hard",
  "old hat",
  "old school",
  "on a hot tin roof",
  "on a roll",
  "on a silver platter",
  "on a tear",
  "on board",
  "on cloud nine",
  "on cue",
  "on demand",
  "on doorstep",
  "on edge",
  "on fumes",
  "on one's feet",
  "on one’s uppers",
  "on pins and needles",
  "on the back burner",
  "on the ball",
  "on the blink",
  "on the brink of",
  "on the cards",
  "on the case",
  "on the crest of a wave",
  "on the dot",
  "on the drawing board",
  "on the horizon",
  "on the line",
  "on the mend",
  "on the money",
  "on the other hand",
  "on the rocks",
  "on the same page",
  "on the tip of my tongue",
  "on thin ice",
  "on top of the world",
  "on your mark",
  "on/to hand",
  "once bitten, twice shy",
  "once in a blue moon",
  "one cannot love and be wise",
  "one foot in the grave",
  "one on one",
  "one stop shop",
  "one too many",
  "one's cake and eat it, too",
  "only game in town",
  "other half",
  "out of action",
  "out of business",
  "out of hand",
  "out of mind",
  "out of one’s depth",
  "out of sight, out of mind",
  "out of steam",
  "out of the blue",
  "out of the closet",
  "out of the corner of your eye",
  "out of the frying pan and into the fire",
  "out of the frying pan into the fire",
  "out on the town",
  "outbreak",
  "outside the lines",
  "over and above",
  "over and out",
  "over and over again",
  "over my dead body",
  "over the coals",
  "over the moon",
  "over the top",
  "over till the fat lady sings",
  "own accord",
  "pack heat",
  "paddle one's own canoe",
  "paddle own canoe",
  "page-turner",
  "pain in the",
  "pain in the neck",
  "paint the town red",
  "par for the course",
  "pardon me",
  "part and parcel",
  "part brass rags",
  "pass away",
  "pass the baton",
  "pass the buck",
  "pass the torch",
  "pass with flying colors",
  "passing fancy",
  "pat on the back",
  "pay as you go",
  "pay lip service",
  "pay the piper",
  "peaches and cream",
  "pecking order",
  "pedal to the metal",
  "pencil something in",
  "penny for your thoughts",
  "penny saved is a penny earned",
  "penny-pinching",
  "pep talk",
  "perfect storm",
  "pet peeve",
  "pick a fight",
  "pick up the slack",
  "pick up the tab",
  "picture is worth 1000 words",
  "picture is worth a thousand words",
  "pie in the sky",
  "piece of cake",
  "piece of work",
  "piece of writing",
  "piece of your mind",
  "pig in a poke",
  "pin money",
  "pin someone down",
  "pinch pennies",
  "pink slip",
  "pipe down",
  "pipe dream",
  "piping hot",
  "pipped to the post",
  "piss and vinegar",
  "pissing contest",
  "pitch in",
  "plain as day",
  "play ball",
  "play by ear",
  "play cat and mouse",
  "play devil's advocate",
  "play hardball",
  "play it by ear",
  "play the percentages",
  "play the ponies",
  "play with fire",
  "play your cards right",
  "playing second fiddle",
  "playing with a full deck",
  "plum job",
  "point blank",
  "point of no return",
  "point out",
  "point the finger",
  "poison pill",
  "poles apart",
  "pop one's clogs",
  "pop one's cork",
  "pop the question",
  "pot calling the kettle black",
  "pound of flesh",
  "pour cold water on",
  "pour oil on troubled waters",
  "powder keg",
  "powder one's nose",
  "practice makes perfect",
  "preach to the choir, preach to the converted",
  "preaching to the choir",
  "pretty penny",
  "prevention is better than cure",
  "price yourself out of the market",
  "prick ears up",
  "prick up ears",
  "prone to",
  "pros and cons",
  "puddle jumper",
  "pull a rabbit out of the hat",
  "pull leg",
  "pull out all the stops",
  "pull someone's leg",
  "pull strings",
  "pull the plug",
  "pull the plug on",
  "pull the wool over eyes",
  "pull your socks up",
  "pull yourself together",
  "puppies and rainbows",
  "puppy dog eyes",
  "puppy love",
  "pure as the driven snow",
  "push the boat out",
  "push the envelope",
  "pushing up daisies",
  "put a sock in it",
  "put a thumb on the scale",
  "put cards on the table",
  "put damper on",
  "put down roots",
  "put in one's two cents",
  "put lipstick on a pig",
  "put one's face on",
  "put out feelers",
  "put someone on the spot",
  "put something on ice",
  "put that in your pipe and smoke it",
  "put the best face on",
  "put the brakes on",
  "put the cart before the horse",
  "put the cat among the pigeons",
  "put the genie back in the bottle",
  "put the pedal to the metal",
  "put up with",
  "put words into someone's mouth",
  "put your foot down",
  "put your foot in it",
  "put your foot in your mouth",
  "put your money where your mouth is",
  "quake in one's boots",
  "quality time",
  "quantum leap",
  "quarter of",
  "quarter past",
  "quarter to",
  "queensberry rules",
  "queer pitch",
  "qui vive",
  "quick as a flash",
  "quick-and-dirty",
  "quote unquote",
  "quote, unquote",
  "rabbit hole",
  "race against time",
  "rack your brains",
  "rags to riches",
  "rain cats and dogs",
  "rain check",
  "rain on someone's parade",
  "rain or shine",
  "rainbow baby",
  "raining cats and dogs",
  "raise one's voice",
  "raise red flags",
  "raise the bar",
  "raise the roof",
  "rake over the ashes",
  "rake over the coals",
  "rank and file",
  "rare bird",
  "rat race",
  "ray of sunshine",
  "raze to the ground",
  "read between the lines",
  "read the riot act",
  "read the tea leaves",
  "rear its ugly head",
  "rearrange the deck chairs on the titanic",
  "recharge one’s batteries",
  "red book",
  "red flag",
  "red herring",
  "red letter day",
  "red meat",
  "red tape",
  "red zone",
  "red-handed",
  "red-light district",
  "reinvent the wheel",
  "rest assured",
  "rest in peace (rip)",
  "riddle me",
  "ride (on) a wave",
  "ride high",
  "riding high",
  "riding shotgun",
  "right as rain",
  "right away",
  "right under one's nose",
  "right-hand man",
  "ring a bell",
  "ring fencing",
  "rip off",
  "rise and shine",
  "rise to the occasion",
  "road to recovery",
  "roaring trade",
  "rob peter to pay paul",
  "rob the cradle",
  "rock bottom",
  "rock the boat",
  "rocket science",
  "roll the dice",
  "roll the dice on",
  "roll with the punches",
  "rome was not built in one day",
  "rome wasn't built in a day",
  "rookie mistake",
  "roots run deep",
  "rotten to the core",
  "rough diamond",
  "round the clock",
  "rub it in",
  "rub someone's nose in",
  "rub something in someone's face",
  "rub the wrong way",
  "rubber-stamp",
  "rule of thumb",
  "rule out",
  "run a tight ship",
  "run across",
  "run amok (amuck)",
  "run around in circles",
  "run for cover",
  "run for their money",
  "run in the family",
  "run into a buzz",
  "run off at the mouth",
  "run out",
  "run out of steam",
  "run the table",
  "running around like a headless chicken",
  "running on fumes",
  "sacred cow",
  "safe bet",
  "salad days",
  "salt in an open wound",
  "salt in the wound",
  "salt on an open wound",
  "salt on the wound",
  "save face",
  "save your breath",
  "saved by the bell",
  "saving for a rainy day",
  "saving grace",
  "say what",
  "scapegoat",
  "scare the living daylights out of someone",
  "scared to death",
  "scaredy-cat",
  "school of hard knocks",
  "scorched earth",
  "scot-free",
  "scrape the barrel",
  "scrape the bottom of the barrel",
  "scratch back",
  "screw loose",
  "screw the pooch",
  "screw up",
  "sea change",
  "seal of approval",
  "sealed with a kiss",
  "second banana",
  "second bite at the cherry",
  "second stringer",
  "second wind",
  "see eye to eye",
  "see the light",
  "seize the day",
  "self-made man",
  "sell like hot cakes",
  "sell like hotcakes",
  "sell someone out",
  "selling point",
  "sense of humour",
  "set in stone",
  "set in ways",
  "set off",
  "set something to music",
  "set the bar",
  "set the bar high",
  "set the bar too high",
  "set the thames on fire",
  "set the world on fire",
  "seventh heaven",
  "shades of grey",
  "shadowban",
  "shake a stick",
  "shake the dust off",
  "shape up or ship out",
  "sharp as a tack",
  "shed light",
  "shell game",
  "shift gears",
  "shifting sands",
  "shilly-shally",
  "shipshape and bristol fashion",
  "shit a brick",
  "shoot from the hip",
  "shoot off one's mouth",
  "shoot oneself in the foot",
  "shoot the breeze",
  "short fuse",
  "shot across the bow",
  "shot in the arm",
  "shoulder a weight off your shoulders",
  "show me an x and i'll show you a y",
  "show one's true colors",
  "show the door",
  "show true colors",
  "show your cards",
  "sick and tired of",
  "sick as a dog",
  "sick as a parrot",
  "sight for sore eyes",
  "silence is golden",
  "silver bullet",
  "silver lining",
  "silver spoon",
  "simmer down",
  "since sliced bread",
  "sing a different tune",
  "sink or swim",
  "sit on",
  "sit on the fence",
  "sit tight",
  "sitting duck",
  "sitting on the fence",
  "sitting pretty",
  "six feet under",
  "six of one, a half dozen of the other",
  "six of one, half a dozen of the other",
  "six ways from sunday",
  "six ways to sunday",
  "sixth sense",
  "skating on thin ice",
  "skeleton in the closet",
  "skid row",
  "skin in the game",
  "slam dunk",
  "slap on the wrist",
  "sleep like a baby",
  "sleep on",
  "sleep tight",
  "sleep with the fishes",
  "slip of the tongue",
  "slip someone a mickey",
  "slippery slope",
  "slow and steady wins the race",
  "slow day",
  "slower than molasses",
  "smack dab",
  "small beer",
  "small fry",
  "small potatoes",
  "smell a rat",
  "smell something fishy",
  "smitten with",
  "smoking gun",
  "snafu",
  "snake oil",
  "sneak peek",
  "snooze, you lose",
  "snow job",
  "snow under",
  "snowball effect",
  "snowed under",
  "so and so",
  "so far so good",
  "so so",
  "so to speak",
  "soak up the sun",
  "sold on",
  "some eggs",
  "something to crow",
  "son of a gun",
  "sore point",
  "sour grapes",
  "southpaw",
  "spanner in the works",
  "spare someone’s blushes",
  "spare the rod and spoil the child",
  "speak of the devil",
  "speak soon",
  "speak too soon",
  "speak up",
  "speak with a plum in one's mouth",
  "spend a fortune",
  "spick and span",
  "spill the beans",
  "spin a yarn",
  "spin one's wheels",
  "spine-tingling",
  "spit into the wind",
  "spitting image",
  "split up",
  "spoiling for a fight",
  "sponger, sponging, sponge off",
  "square peg in a round hole",
  "square the circle",
  "stab in the back",
  "stab someone in the back",
  "stack the deck",
  "stalking horse",
  "stand a chance",
  "stand in a good stead",
  "stand on one's own two feet",
  "stand one's ground",
  "stand out",
  "stand someone in good stead",
  "stand your ground",
  "stands to reason",
  "start from scratch",
  "start with a clean slate",
  "status quo",
  "stay up",
  "steal someone's thunder",
  "steal thunder",
  "stem the tide",
  "step up one's game",
  "step up to the plate",
  "step up your game",
  "stick it to the man",
  "stick your nose into something",
  "stick-in-the-mud",
  "sticker shock",
  "sticky fingers",
  "sticky wicket",
  "stiff upper lip",
  "stiff-necked",
  "still waters run deep",
  "stir up a hornet’s nest",
  "stone’s throw",
  "stop by",
  "storm in a teacup",
  "stormy relationship",
  "straight arrow",
  "straight from the horse’s mouth",
  "straight shooter",
  "straighten up and fly right",
  "strain at a gnat and swallow a camel",
  "strike a chord",
  "strike while the iron is hot",
  "strive to do",
  "stumbling block",
  "sugar daddy",
  "suited and booted",
  "sure-fire",
  "surgical strike",
  "swan song",
  "sweep under the carpet",
  "sweep under the rug",
  "sweet dreams",
  "sweet tooth",
  "sweeten the deal",
  "sweeten the pot",
  "swim against the tide",
  "swim with sharks",
  "swim with the fishes",
  "swing a dead cat",
  "swing for the fences",
  "sword of damocles",
  "take a back seat",
  "take a beating",
  "take a cue from",
  "take a flyer",
  "take a gander",
  "take a hike",
  "take a leaf out of someone’s book",
  "take a powder",
  "take a rain check",
  "take aback",
  "take breath away",
  "take care",
  "take effect",
  "take five",
  "take for a ride",
  "take for granted",
  "take heart",
  "take into account",
  "take it easy",
  "take it on the chin",
  "take it or leave it",
  "take it to the bank",
  "take it up/down a notch",
  "take it with a grain of salt",
  "take name in vain",
  "take on",
  "take over",
  "take someone to task",
  "take stock",
  "take ten",
  "take the bull by the horns",
  "take the cake",
  "take the edge off",
  "take the fifth",
  "take the gloves off",
  "take the high road",
  "take the shine off",
  "take the starch out",
  "take the wind out of someone's sails",
  "take turns",
  "take up the cudgels",
  "take with a grain of salt",
  "take your life in your hands",
  "take your medicine",
  "take your time",
  "takes two to tango",
  "tall poppy",
  "tall story",
  "tar with the same brush",
  "taste of own medicine",
  "taste of your own medicine",
  "tasted blood",
  "tea in china",
  "teach an old dog new tricks",
  "tear one's hair out",
  "tear-jerker",
  "tear-jerking",
  "tee many martoonies",
  "tell it to the marines",
  "tempest in a teapot",
  "ten a penny",
  "ten to one",
  "test the waters",
  "thank god it's friday",
  "that ship has sailed",
  "that's a stretch",
  "that's all she wrote",
  "that's music to my ears",
  "that's the last straw",
  "that’s the way the cookie crumbles",
  "the 8 ball",
  "the 8-ball",
  "the apple doesn’t fall far from the tree",
  "the apple never falls far from the tree",
  "the ball is in your court",
  "the ball rolling",
  "the benefit of the doubt",
  "the best is yet to come",
  "the best of both worlds",
  "the best thing since sliced bread",
  "the better of",
  "the bigger they are the harder they fall",
  "the birds and the bees",
  "the brains behind",
  "the buck stops here",
  "the bull by the horns",
  "the calm before the storm",
  "the cat is out of the bag",
  "the cherry on the cake",
  "the cobwebs away",
  "the corridors of power",
  "the course of true love never did run smooth",
  "the devil is beating his wife",
  "the devil is in the detail",
  "the die is cast",
  "the drawing board",
  "the early bird catches the worm",
  "the eight ball",
  "the eight-ball",
  "the eleventh hour",
  "the hots for",
  "the kool-aid",
  "the last straw",
  "the last thing i want",
  "the lion’s den",
  "the other side of the coin",
  "the pen is mightier than the sword",
  "the powers that be",
  "the real mccoy",
  "the rose that grew from concrete",
  "the smell of an oily rag",
  "the squeaky wheel gets the grease",
  "the story has legs",
  "the time is ripe",
  "the time of your life",
  "the walls have ears",
  "the walls have ears we",
  "the whole enchilada",
  "the whole nine yards",
  "the whole shebang",
  "the world is your oyster",
  "there are clouds on the horizon",
  "there are other fish in the sea",
  "there but for the grace of god go i",
  "there's a method to his madness",
  "there's more than one way to skin a cat",
  "there's no such thing as a free lunch",
  "there’s no place like home",
  "there’s no such thing as a free lunch",
  "there’s no time like the present",
  "thin on the ground",
  "think big",
  "think outside the box",
  "think tank",
  "third degree",
  "third rail",
  "third time's a charm",
  "third time’s a charm",
  "third wheel or fifth wheel",
  "thirty-thousand-foot view",
  "this is not your father's",
  "those three little words",
  "those who live in glass houses shouldn't throw stones",
  "three cheers",
  "three sheets to the wind",
  "through the grapevine",
  "through the nose",
  "through thick and thin",
  "throw a wet blanket on",
  "throw a wrench into",
  "throw caution to the wind",
  "throw caution to the winds",
  "throw down the gauntlet",
  "throw down the gauntlet",
  "throw dust in eyes",
  "throw elbows",
  "throw in the towel",
  "throw someone for a loop",
  "throw someone under the bus",
  "throw the baby out with the bath water",
  "throw the baby out with the bathwater",
  "throw the book at",
  "throw the fight",
  "throw the game",
  "throw the match",
  "throw to the wolves",
  "throw under the bus",
  "thumbs-up",
  "tick off",
  "tickle someone’s fancy",
  "tickled pink",
  "tide over",
  "tie the knot",
  "tight-lipped",
  "tighten belt",
  "tighten the screws",
  "til the cows come home",
  "till doomsday",
  "time after time",
  "time and again",
  "time flies",
  "time flies when you're having fun",
  "time is money",
  "time puts everything in its place",
  "time will tell",
  "tip of the iceberg",
  "tip one's hand",
  "tit for tat",
  "tlc",
  "to a t",
  "to be a peach",
  "to be smitten with someone",
  "to be someone's one and only",
  "to be the apple of someone's eye",
  "to bear fruit",
  "to carpool",
  "to each his own",
  "to get bent out of shape",
  "to have butterflies in your stomach",
  "to have one for the road",
  "to heart’s content",
  "to make matters worse",
  "to pay an arm and a leg",
  "to pop one's cherry",
  "to pull someone's leg",
  "to run hot and cold",
  "to see the back of",
  "to square one",
  "to terms with",
  "to the cleaners",
  "to the core",
  "to the letter",
  "to the nines",
  "to the quick",
  "to the salt mine",
  "to the salt mines",
  "toe the line",
  "tone-deaf",
  "tongue in cheek",
  "tongue-in-cheek",
  "too busy fighting alligators",
  "too close for comfort",
  "too close to call",
  "too many chiefs",
  "too many cooks",
  "too many cooks spoil the broth",
  "toot one’s own horn",
  "toot your own horn",
  "top banana",
  "top dog",
  "topsy-turvy",
  "toss a wrench",
  "toss-up",
  "touch base",
  "touch off",
  "touch one's heart",
  "touch water",
  "touch wood",
  "tough cookie",
  "tough sledding",
  "train wreck",
  "trash talk",
  "tread water",
  "treasure trove",
  "treat like dirt",
  "trial balloon",
  "trip the light fantastic",
  "trouble shared is trouble halved",
  "troubled waters",
  "turn a blind eye",
  "turn a deaf ear",
  "turn against",
  "turn aside",
  "turn back on",
  "turn down a glass",
  "turn on a dime",
  "turn out",
  "turn over a new leaf",
  "turn someone inside out",
  "turn something on its head",
  "turn the clock back",
  "turn the corner",
  "turn the tables",
  "turn turtle",
  "turnabout is fair play",
  "turning point",
  "twenty-four seven",
  "twenty-three skidoo",
  "twist arm",
  "twist of fate",
  "twist the knife",
  "two a penny",
  "two cents",
  "two heads are better than one",
  "two peas in a pod",
  "two wrongs don’t make a right",
  "u turn",
  "u-turn",
  "ugly duckling",
  "uncle tom cobley and all",
  "under a cloud",
  "under fire",
  "under no illusions",
  "under siege",
  "under someone's spell",
  "under the auspices",
  "under the impression",
  "under the radar",
  "under the table",
  "under the weather",
  "under wraps",
  "university of life",
  "unring the bell",
  "until the cows come home",
  "until you're blue in the face",
  "up a blind alley",
  "up a creek",
  "up a creek without a paddle",
  "up and about",
  "up for grabs",
  "up in arms",
  "up in the air",
  "up the ante",
  "up the duff",
  "up the wall",
  "up to no good",
  "up to one's neck",
  "up to scratch",
  "up to snuff",
  "up to the mark",
  "up to the nines",
  "upper crust",
  "upper hand",
  "ups and downs",
  "upset the apple cart",
  "upset the applecart",
  "upside down",
  "use one's head",
  "use your loaf",
  "vale of tears",
  "vale of years",
  "van gogh’s ear for music",
  "variety is the spice of life",
  "veg out",
  "vent spleen",
  "vice versa",
  "vicious circle",
  "victory lap",
  "vim and vigor",
  "viper in bosom",
  "virgin territory",
  "volte face",
  "vote with one's feet",
  "wag the dog",
  "waiting in the wings",
  "waiting time",
  "waka-jumping",
  "wake up and smell the coffee",
  "wake up on the wrong side of the bed",
  "walk on eggshell",
  "walk on eggshells",
  "walk out on",
  "walk the plank",
  "walking on air",
  "wandering eye",
  "wane",
  "ward off",
  "warm-hearted",
  "warn off",
  "wash dirty linen in public",
  "wash hands of",
  "wash your hands of",
  "waste not, want not",
  "watchdog",
  "water under the bridge",
  "watering hole",
  "way around",
  "way forward",
  "we see eye to eye",
  "weak in the knees",
  "wear heart on sleeve",
  "wear off, out",
  "weather the storm",
  "weekend warrior",
  "welch on a deal",
  "well begun is half done",
  "well read",
  "well-to-do",
  "welsh on a deal",
  "wet behind the ears",
  "wet blanket",
  "wet your whistle",
  "whale of a time",
  "what are you up to",
  "what do you make of",
  "what goes around comes around",
  "when hell freezes over",
  "when in doubt, leave it out",
  "when in rome, do as the romans",
  "when in rome, do as the romans do",
  "when it rains it pours",
  "when it rains, it pours",
  "when life gives you lemons",
  "when pigs fly",
  "when the chips are down",
  "when the going gets tough, the tough get going",
  "when the rubber",
  "where the rubber",
  "where the rubber meets the road",
  "where there's a will, there's a way",
  "where there's smoke, there's fire",
  "where there’s a will, there’s a way",
  "whet one’s appetite",
  "whisper sweet nothings",
  "whistle in the dark",
  "whistle past the graveyard",
  "whistling dixie",
  "white elephant",
  "white lie",
  "white paper",
  "whole nine yards",
  "wide berth",
  "wild and woolly",
  "wild card",
  "wild goose chase",
  "willy-nilly",
  "win the day",
  "wind up",
  "window dressing",
  "window shop",
  "wine and dine",
  "witch hunt",
  "with a bang",
  "with a grain of salt",
  "with bells on",
  "within earshot",
  "without further ado",
  "word of mouth",
  "worm in the apple",
  "worn to a frazzle",
  "worth your salt",
  "would put legs under a chicken",
  "wouldn't be caught dead",
  "wouldn't put it past",
  "wouldn’t be caught dead",
  "wouldn’t dream of",
  "wrap in cotton wool",
  "wrap one’s head around",
  "wrap your head around something",
  "writing on the wall",
  "written all over it",
  "x factor",
  "x marks the spot",
  "year dot",
  "year in, year out",
  "year-round",
  "yellow streak",
  "yes man",
  "yesterday’s man",
  "yoke around one’s neck",
  "you are what you eat",
  "you bet",
  "you can lead a horse to water but you can’t make him drink",
  "you can say that again",
  "you can’t make an omelette without breaking a few eggs",
  "you can’t take it with you",
  "you can’t teach an old dog new tricks",
  "you can’t win them all",
  "you know better than that",
  "you rock",
  "you wish",
  "young at heart",
  "young blood",
  "young turk",
  "your call",
  "your guess is as good as mine",
  "your sins will find you out",
  "yours truly",
  "you’re on",
  "you’ve got to be kidding",
  "you’ve made your bed",
  "zenith of career or life",
  "zero hour",
  "zero in on",
  "zero tolerance",
  "zero-sum game",
  "zig when one should be zagging",
  "zip it",
  "zip one's lip",
  "zip past",
  "zip your lip",
  "zone out",
  "zonk out",
  "zoom away",
  "zoom off",
]

In [ ]:
# @title **Idioms**
"""Has all idioms and manages identifying and reporting those found"""


class MyIdioms:
    """Class to handle management of idioms"""
    # TODO need to account for no punctuation in my string i am checking for
    # TODO need to check for no punctuation in the string checking
    # TODO need to make sure that line breaks aren't interrupting the phrase
    # TODO need to find permutations of the idioms
    # TODO have duplicates in there with different types of punctuation

    def __init__(self):
        self.idiom_report = []
        self.idioms = idiom_list
        print(f"DEBUG:\t{len(self.idioms)} idioms loaded in for checking.")

    def run_line(self, line, line_num):
        """runs each discrete line against the idioms"""
        _ = [self.idiom_report.append(f"\t\item line {line_num:,}:  ``{i}''") for i in self.idioms if i.lower() in line]

    def report_out(self):
        """Generate report of findings against idioms"""

        if len(self.idiom_report) != 0:
            self.idiom_report.sort()
            finds = r"".join([f"\n{i}" for i in self.idiom_report])
            to_report = f'Discovered {len(self.idiom_report):,} idioms:\n'
            to_report += r"\begin{itemize}[itemsep=0ex]"
            to_report += f" {finds}"
            to_report += r"\end{itemize}"
            return to_report

        return '\nIdiom Report complete, no discrepancies.\n'


In [ ]:
# @title **Replace**
replace_dict = [
	[" /", "/"],
	[" \n", "\n"],
	[" \r", "\r"],
	[" ^p", "^p"],
	["..", "."],
	["/ ", "/"],
	["required to", "must"],
	["who have", "with"],
	["Kirkland AFB", "Kirtland AFB"],
	["Provide an update brief to the", "brief"],
	["Provided an update brief to the", "briefed"],
	["Washington D.C.", "Washington, D.C."],
	["Washington DC", "Washington, D.C."],
	["Washington DC.", "Washington, D.C."],
	["Washington, DC", "Washington, D.C."],
	["Washington, DC.", "Washington, D.C."],
	["are required to", "must"],
	["^+", "-"],
	["^=", "-"],
	["^l", "^p"],
	["^s", " "],
	["add additional", "add"],
	["adhoc", "ad hoc"],
	["all things considered", ""],
	["along with", "with"],
	["also continue", "continue"],
	["amongst", "among"],
	["as a matter of fact", ""],
	["as far as I'm concerned", ""],
	["as well as", "and"],
	["believe", "assess"],
	["blacklist", "blocklist"],
	["blackmail", "extort"],
	["can't", "cannot"],
	["close proximity", "proximity"],
	["completely unanimous", "unanimous"],
	["conducted research", "researched"],
	["continually developing", "developing"],
	["continued to aggressively work", "worked"],
	["continued working with", "worked with"],
	["could not agree", "disagreed"],
	["could potentially", "could"],
	["couldn't agree", "disagreed"],
	["current draft", "draft"],
	["current status", "status"],
	["currently working", "working"],
	["de-brief", "debrief"],
	["decided", "chose"],
	["did state", "stated"],
	["didn't", "did not"],
	["doesn't", "does not"],
	["don't", "do not"],
	["drew a diagram", "diagrammed"],
	["due to the fact that", "because"],
	["earlier than", "before"],
	["exactly the same", "the same"],
	["firm up", "finalize"],
	["for all intents and purposes", ""],
	["for the most part", ""],
	["further processing", "processing"],
	["future planning", "planning"],
	["future plans", "plans"],
	["had identified", "identified"],
	["had previously", "previously"],
	["has a tendency to", "tends"],
	["has identified", "identified"],
	["have a tendency to", "tends"],
	["how's", "how is"],
	["in a manner of speaking", ""],
	["in a very real sense", ""],
	["in an effort to", "to"],
	["in between", "between"],
	["in compliance with", "in accordance with"],
	["in depth", "thorough"],
	["in effort to", "to"],
	["in efforts to", "to"],
	["in her efforts to", "to"],
	["in his efforts to", "to"],
	["in order for", "for"],
	["in order to", "to"],
	["in preparation for", "for"],
	["in regards to", "regarding"],
	["in the event that", "if"],
	["in the process of", ""],
	["in their efforts to", "to"],
	["in-depth", "thorough"],
	["isn't", "is not"],
	["it's", "it is"],
	["key stakeholders", "stakeholders"],
	["made a diagram", "diagrammed"],
	["made the decision", "decided"],
	["main focus", "focus"],
	["members of the team", "team members"],
	["more streamlined", "streamlined"],
	["move forward", "proceed"],
	["not able", "unable"],
	["not adequate", "inadequate"],
	["not sufficient", "insufficient"],
	["offered explanation", "explained"],
	["one and the same", "the same"],
	["one in the same", "the same"],
	["one of the", "a"],
	["performed an analysis", "assessed"],
	["performed an initial analysis", "assessed"],
	["primary focus", "focus"],
	["provided training to", "trained"],
	["provided with", "given"],
	["put together", "compiled"],
	["quick", ""],
	["reached out to", "engaged"],
	["repeat again", "repeat"],
	["revise and update", "update"],
	["spearheaded", "led"],
	["started off", "began"],
	["such as", "like"],
	["take action on", "act on"],
	["telecom", "telecon"],
	["that's", "that is"],
	["the addition of", "adding"],
	["then discussed", "discussed"],
	["then proceeded", "proceeded"],
	["then provided", "provided"],
	["those personnel", "personnel"],
	["to discuss agenda items previously listed", ""],
	["took place", "occurred"],
	["travelled", "traveled"],
	["up to date", "current"],
	["update and revise", "update"],
	["updated existing", "updated"],
	["utilize", "use"],
	["utilizing", "using"],
	["vast majority", "majority"],
	["vast", ""],
	["was also", "was"],
	["was in agreement", "agreed"],
	["were able to meet", "met"],
	["were in agreement", "agreed"],
	["what'll", "what will"],
	["what're", "what are"],
	["what's", "what is"],
	["when's", "when is"],
	["where's", "where is"],
	["whitelist", "safelist"],
	["who's", "who is"],
	["why's", "why is"],
	["within", "in"],
	["wont", "will not"],
	["you've got", "you have"],
	["whenever", "when"],
	["to ensure", "ensure"],
]


In [ ]:
# @title **Reword**
reword_dict = [
	["  ", "Double space may indicate inconsistent use of single and double space; if reviewing a plain text file that uses spaces for syntax, this flag may be unnecssary."],
	[" also ", "Usually working against you"],
	[" of the ", "Opportunity to streamline subject sentence placement"],
	["(aka ", "Is this supposed to be 'also known as'? not formal enough"],
	["(eg", "This is meant to check for erroneous use of e.g., but will also flag first words starting with 'eg'"],
	["(ie", "Meant to check for improper uses of 'i.e.', but also indicate that unless it is a finite list, it likely needs to be 'e.g.'"],
	[", aka ", "Is this supposed to be 'also known as'? not formal enough"],
	[".  ", "Double space following a period. If double space if the standard in the docuemnt and the found count for this value and the next match, it is likely all spaces are standardized."],
	[".)", "Rarely appropriate to have a period inside an ellipses."],
	["?", "Unless a part of a quote or action item, most formal writing will not be posing questions."],
	["In the end", "No need to preface the message, just say it - remove this."],
	["Security Center", "If this is referencing the software tool, needs to be one word SecurityCenter"],
	["a.k.a.", "No."],
	["actually", "Usually fluff, avoid when able"],
	["additionally", "Try to avoid these bridges"],
	["aka, ", "Is this supposed to be 'also known as'? not formal enough"],
	["also, ", "Usually working against you"],
	["as required", "Chances are it is required or was left over boilerplate - is this value added?"],
	["basically", "Usually fluff, avoid when able"],
	["briefly", "How important is speed to the sentence? that it occurred should be the extent of its noteworthiness"],
	["both", "Typically unneeded given that you're about to provide them inclusively to the caveats/burdens"],
	["by to assist", "If are saying you are standing by to assist, you are saying 'i am willing to do my job', delete."],
	["carefully", "Are you talking about a nurse taking care of someone? if not, kill it (preferred) or reword"],
	["commercial", "If this is talking about airfare, the type is unimportant"],
	["conversed", "Spoke"],
	["course of action", "Try replacing with 'option(s)'"],
	["courses of action", "Try replacing with 'option(s)'"],
	["create new, ", "Generally the creation is innately 'new' is it not? Recommend removing 'new'"],
	["crew rest", "Unless this contract is providing CAT/CWD and you are aircrew, you do not get crew rest"],
	["definite", "Very few things are definite, avoid"],
	["develop and refine", "Same thing, pick one"],
	["development and refinement", "Same thing, pick one"],
	["dramatic", "Drama belongs in the theater. No."],
	["e.g. ", "Make sure this is used correctly - looking for e.g.,"],
	["essentially", "No."],
	["etc.", "Do not leave it open ended, be explicit or remove it"],
	["example", "Use the word in a narrative, parenthetically use e.g."],
	["excellent", "Typically subjective"],
	["extreme", "Very few things in formal writing are genuinely extreme, remove or reword."],
	["first priority", "Generally 'priority' stands alone without a qualifier, can probably delete 'first'"],
	["for the purpose of", "Flowery rhetoric, can be replaced with 'as' or something similar in most cases"],
	["further", "Try to avoid these bridges"],
	["go tdy", "Contractors travel, military and government go TDY."],
	["great", "Typically subjective"],
	["has stated", "Both are past tense, pick one - probably 'stated'"],
	["health", "If you are talking about a system, it lacks health, it is not alive, use a more formal term"],
	["held at", "Can this just be 'at'?"],
	["hotel", "If a trip report, do not talk about when you left your hotel nor when you left the site to return to it"],
	["i.e.", "Do you mean to say e.g.?"],
	["if required", "Vague or leftover boilerplate, rarely value added?"],
	["important to note", "By writing what follows this, you are noting it... no need to preface it as such - fluff"],
	["in accordance with", "Can this just be IAW?"],
	["in addition", "Avoid when able"],
	["in fact", "No. Just say it, that will make it fact enough"],
	["in my opinion", "No place in formal writing"],
	["in support of", "Can this be 'for'?"],
	["in that it", "Fluff"],
	["insightful", "Unless you're talking about someone who is world renowned for being a visionary, this is hyperbole"],
	["many", "Rarely adds value"],
	["mission complete", "We are contractors, trip reports will not state a mission is complete"],
	["more", "Usually fluff, avoid when able"],
	["multiple", "Typically superfluous"],
	["myself", "Avoid making it personal."],
	["necessary", "Usually fluff, avoid when able"],
	["needless to say", "No. If it is truly needless to say, don't say it. If you need to say it, remove this."],
	["not limited", "Make sure list doesn't end with 'and others' or 'etc.' as the preface says there's more than listed"],
	["not only", "This almost always hurts your message, cut to the chase, say what you mean"],
	["noteworthy", "By writing what follows this, you are noting it... no need to preface it as such - fluff"],
	["numerous", "Typically superfluous"],
	["of that nature", "Generally superfluous."],
	["overall", "Too vague, colloquial"],
	["overarching", "This word rarely adds value"],
	["perform", "Performances belong in the theater"],
	["personally", "No place in formal writing"],
	["picked up", "Would 'received' be more fitting? Picked up perceived as informal"],
	["possible option", "Option is inherently not definitive"],
	["potential option", "Option is inherently not definitive"],
	["preplanned", "If you've planned, it is inherently 'preplanned', review"],
	["reach back", "Better stated as 'Remotely'? The term is military jargon to say the same thing as remotely."],
	["reach-back", "Better stated as 'Remotely'? The term is military jargon to say the same thing as remotely."],
	["reachback", "Better stated as 'Remotely'? The term is military jargon to say the same thing as remotely."],
	["really", "Usually fluff, avoid when able"],
	["recommend", "A shorter word would be suggest (in any tense), consider changing."],
	["refine and develop", "Same thing, pick one"],
	["regarding", "Try replacing with 'on'. Won't always work, but when it does, big space saver."],
	["several", "Rarely value added"],
	["shall", "Unless this is a PWS, chances are we want to remove this"],
	["specialized", "Probably fluff, usually used in concert with another term that is inherently specialized"],
	["specifically", "Just say it, that you single it out will be innately specific"],
	["subsequently", "Usually fluff, avoid when able"],
	["talk through", "Can you just say 'discuss'?"],
	["tenfold", "Unless a literal ten time increase, don't use this avoid misinterpretation of a legitimate measure"],
	["tentative", "Generally a sentence can stand as well by removing this."],
	["thanking everyone for attending", "Someone's appreciation for participation is not value added."],
	["the ability to", "Can this just be 'to'?"],
	["thoughtful", "Usually working against you"],
	["to the left", "If this is talking about time, it is colloquial, reword"],
	["to the right", "If this is talking about time, it is colloquial, reword"],
	["upcoming", "Generally superfluous."],
	["varied", "Typically superfluous"],
	["various", "Typically superfluous"],
	["varying", "Typically superfluous"],
	["very", "Usually fluff, avoid when able"],
	["work closely", "Proximity unneeded"],
	["worked closely", "Proximity is rarely worth noting, 'working' should be sufficient"],
	["working closely", "Proximity is rarely worth noting, 'working' should be sufficient"],
	["writ large", "Generally superfluous."],
	["yes or no", "If this is a template and is seeking an answer, remove this after you've entered an answer"],
	["that", "'that' is a superfluous word that rarely adds value, can be removed for clarity and to be concise"],
]


In [ ]:
# @title **Main**
"""Core logic, run all files and generates reports"""

from idioms import MyIdioms
from acronyms import Acronyms
from clarity import Clarity
from datetime import datetime
from file_handling import FileGather

# TODO full review of the comments/recommendations

FileLib = FileGather()

for in_file in FileLib.parsing_dicts:
	idm = MyIdioms()
	_ = [idm.run_line(str(line).lower(), x) for x, line in enumerate(in_file['narrative'])]
	acr = Acronyms()
	_ = [acr.run_line(line, x) for x, line in enumerate(in_file['narrative'])]
	clr = Clarity()
	clr.doc = [str(i) for i in in_file['narrative']]

	latex_file = f'Output/[Report]_{in_file["file"].split(".")[0]}-{datetime.now().strftime("%Y%m%d-%H%M%S")}.tex'

	with open(latex_file, "w") as latex_file:
		# Top content / globals / document style
		latex_file.write("\\documentclass{article}\n\\usepackage[utf8]{inputenc}\n")
		latex_file.write("\\usepackage{enumitem}\n\\usepackage{multicol}\n")  # To customize LaTeX lists
		latex_file.write(r"\usepackage[margin = 0.5 in, bottom = 1 in]{geometry}")
		latex_file.write("\n\\usepackage{fancyhdr}\n")  # To customize LaTeX header/footer
		latex_file.writelines([f"{i}\n" for i in [r"\textwidth=166mm", r"\textheight 23.0cm", r"\oddsidemargin= -2.4mm", r"\evensidemargin= -2.4mm", r"\topmargin= -17.4mm"]])
		latex_file.write(r"\begin{document}" + "\n")  # Start visible document space
		latex_file.write(r"\pagestyle{fancy}" + "\n" + r"\rhead{" + in_file['file'] + "}\n" + r"\lfoot{Report date: \today}" + "\n" + r"\rfoot{\thepage}" + "\n" + r"\cfoot{}")
		latex_file.write("\n\section {Summary}\n")
		latex_file.write(f"Analyzed {len(clr.doc)} lines of text.\n")

		latex_file.write("\section {Idiom Report}\n")
		latex_file.write(idm.report_out())

		latex_file.write("\n\section {Acronym Report}\n")
		latex_file.write(acr.report_out())

		clr.report_out()  # Generates the content of the following two sections
		latex_file.write("\n\section {Reword Report}\n")
		latex_file.write(clr.rew_summ)  # Populates content from the rewording
		latex_file.write("\n\section {Replace Report}\n")
		latex_file.write(clr.repl_summ)  # Populates content from the replacing

		latex_file.write("\end{document}\n")
